In [0]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize #transform json data into pd df
import json

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [0]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')

In [0]:
n_List = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    n_List.append(row.text)

In [0]:
df = pd.DataFrame({"Neighborhood": n_List})
df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [0]:
df.shape

(71, 1)

In [0]:
import sys
!{sys.executable} -m pip install geocoder
import geocoder
from geopy.geocoders import arcgis as arcgis

     |████████████████████████████████| 102kB 3.9MB/s 


In [0]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
coords[:5]

[[3.0576900000000364, 101.74388000000005],
 [3.148494115588384, 101.69672876508707],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005]]

In [0]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [0]:
address = 'Kuala Lumpur, Malaysia'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [0]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [0]:
# define Foursquare Credentials and Version and explore the neighborhoods
CLIENT_ID = 'PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H' # your Foursquare ID
CLIENT_SECRET = 'L2B5GO3BHAZPAS21BWWX2K0XMBCMWFW3QS5DDGLVPCJV0VYB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H
CLIENT_SECRET:L2B5GO3BHAZPAS21BWWX2K0XMBCMWFW3QS5DDGLVPCJV0VYB


In [0]:
#Top 100 venues that are within a radius of 2k meters
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
venues_df = pd.DataFrame(venues,columns=['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory'])
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant


In [0]:
venues_df.groupby(["Neighborhood"]).count() #Number of venues returned for each neighbor remember max limit was 100

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
...,...,...,...,...,...,...
Taman Tun Dr Ismail,100,100,100,100,100,100
Taman U-Thant,100,100,100,100,100,100
Taman Wahyu,100,100,100,100,100,100


In [0]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))  #unique categories from all venues

There are 306 uniques categories.


In [0]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7084, 307)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,...,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# group rows by neighborhood taking the mean of the frequency of occurrence of each category
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(kl_grouped.shape)
kl_grouped

(71, 307)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,...,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Alam Damai,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.05,0.01,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.02,0.0,0.0,0.01,0.0,0.00,0.01,...,0.02,0.00,0.00,0.01,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.01,0.00,0.00,0.00,0.0
1,"Ampang, Kuala Lumpur",0.00,0.01,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.01,0.0,0.0,0.00,0.0,0.01,0.00,0.02,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.01,0.0,0.0,0.01,0.01,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
2,Bandar Menjalara,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.02,0.01,0.0,0.0,0.01,0.0,0.01,0.00,0.01,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.03,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.03,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
3,Bandar Sri Permaisuri,0.01,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.00,0.04,0.00,0.01,0.00,0.02,0.0,0.01,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.0,0.0,0.01,0.0,0.00,0.00,0.00,0.00,0.02,0.0,0.0,0.00,0.0,0.00,0.00,...,0.01,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.01,0.00,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
4,Bandar Tasik Selatan,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.01,0.00,0.02,0.0,0.0,0.01,0.0,0.00,0.04,...,0.01,0.00,0.00,0.00,0.01,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.03,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Taman Tun Dr Ismail,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.04,0.00,0.01,0.00,0.00,0.0,0.0,0.00,0.0,0.01,0.01,0.03,0.00,0.00,0.0,0.0,0.00,0.0,0.01,0.04,...,0.01,0.00,0.00,0.00,0.00,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.0,0.0,0.00,0.00,0.01,0.0,0.01,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
67,Taman U-Thant,0.00,0.00,0.00,0.0,0.01,0.00,0.02,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.01,0.03,0.00,0.00,0.00,0.0,0.0,0.01,0.0,0.01,0.00,0.03,0

**Create a new DataFrame for Shopping Mall data only**

In [0]:
df_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
df_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.01
2,Bandar Menjalara,0.01
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.01


Cluster Neighborhoods


In [0]:
# set number of clusters
kclusters = 3

kl_clustering = df_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 0, 2, 1, 1, 0, 2], dtype=int32)

In [0]:
kl_clustering.head()

,Shopping Mall
0,0.00
1,0.01
2,0.01
3,0.00
4,0.01


In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = df_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [0]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,2
1,"Ampang, Kuala Lumpur",0.01,0
2,Bandar Menjalara,0.01,0
3,Bandar Sri Permaisuri,0.00,2
4,Bandar Tasik Selatan,0.01,0


In [0]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,2,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.01,0,3.148494,101.696729
2,Bandar Menjalara,0.01,0,3.190350,101.625450
3,Bandar Sri Permaisuri,0.00,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.01,0,3.072750,101.714610


In [0]:
# sort the results by Cluster Labels
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.01,0,3.203910,101.737190
23,Dang Wangi,0.01,0,3.156685,101.698077
51,Shamelin,0.02,0,3.124570,101.735970
26,Happy Garden,0.01,0,3.201630,101.721070
27,"Jalan Cochrane, Kuala Lumpur",0.02,0,3.132903,101.724679
...,...,...,...,...,...
40,Miharja,0.00,2,3.147890,101.694050
42,Pantai Dalam,0.00,2,3.094760,101.667470
45,Salak South,0.00,2,3.081020,101.697240
29,Jinjang,0.00,2,3.209500,101.658740


In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the Clusters

#### Cluster 0

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.01,0,3.203910,101.737190
23,Dang Wangi,0.01,0,3.156685,101.698077
51,Shamelin,0.02,0,3.124570,101.735970
26,Happy Garden,0.01,0,3.201630,101.721070
27,"Jalan Cochrane, Kuala Lumpur",0.02,0,3.132903,101.724679
28,Jalan Duta,0.01,0,3.180025,101.677834
60,Taman Melati,0.01,0,3.223570,101.723990
32,Kampung Datuk Keramat,0.01,0,3.166400,101.730460
21,Damansara Town Centre,0.01,0,3.136442,101.690296
58,Taman Ibukota,0.01,0,3.212160,101.715400


#### Cluster 1

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
41,Mont Kiara,0.03,1,3.165320,101.652430
6,Bangsar,0.05,1,3.129200,101.678440
37,Lembah Pantai,0.04,1,3.121202,101.663899
7,Bangsar Park,0.05,1,3.129200,101.678440
67,Taman U-Thant,0.05,1,3.157700,101.724520
43,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070
22,"Damansara, Kuala Lumpur",0.03,1,3.141906,101.679678
31,"Kampung Baru, Kuala Lumpur",0.04,1,3.165460,101.710280
30,KL Eco City,0.04,1,3.117130,101.673840
11,Brickfields,0.04,1,3.129160,101.684060


#### Cluster 2

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
53,Sri Petaling,0.0,2,3.072600,101.682520
68,Taman Wahyu,0.0,2,3.222400,101.671730
56,Taman Connaught,0.0,2,3.082690,101.736890
63,Taman P. Ramlee,0.0,2,3.193940,101.705730
61,Taman Midah,0.0,2,3.093590,101.728370
59,Taman Len Seng,0.0,2,3.069080,101.742870
65,Taman Taynton View,0.0,2,3.087070,101.736810
55,Taman Bukit Maluri,0.0,2,3.200660,101.633370
62,Taman OUG,0.0,2,3.210050,101.634508
0,Alam Damai,0.0,2,3.057690,101.743880


###Quick Result


*   Mostly shopping malls are situated in cluster 1 ranging from 3 to 5.
*   Cluster 1 has highest competition. 
*   Cluster 2 has no malls and hence no competition. 

